# Planetary Spectrum Generator Atmospheric Files

In [ ]:
import os
import glob
import numpy as np
import scipy
import astropy.table
import astropy.io.fits
import astropy.units

import matplotlib.pyplot as plt

import lezargus


%matplotlib Qt

In [ ]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/atmosphere_psg/", exist_ok=True)

[Lezargus] 2024-04-29T19:17:24     INFO -- Configuration file C:\Users\psmdi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lezargus\configuration.yaml was loaded and applied.
[Lezargus] 2024-04-29T19:17:24     INFO -- The temporary log file C:\Users\psmdi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lezargus\lezargus_0563b6e1341d416ead3f5c343d1192ec.log is currently in-use, we defer  deletion until the next load.
[Lezargus] 2024-04-29T19:17:24     INFO -- The temporary log file C:\Users\psmdi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lezargus\lezargus_4b30348bd95546dca7171fc9afac1ffb.log is currently in-use, we defer  deletion until the next load.
[Lezargus] 2024-04-29T19:17:24     INFO -- The temporary log file C:\Users\psmdi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lezargus\lezargus_54945b9ed1504e4497ac81bfc889d50b.log is currently in-use, we defer  deletion until the next load.
[Lezargus] 2024-04-29T19:17:24     INFO -- The t

In [42]:
def get_zenith_angle_from_filename(filename: str) -> int:
    """The filename stores the zenith angle, get it."""
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filename
    )
    zenith_angle = int(basename.split("_")[3].removeprefix("za"))
    return zenith_angle


def get_pwv_from_filename(filename: str) -> float:
    """The filename stores the precipitable water vapor, get it."""
    basename = lezargus.library.path.get_filename_without_extension(
        pathname=filename
    )
    pwv_str = basename.split("_")[4]
    pwv = float(
        pwv_str.removeprefix("h2o").removesuffix("pwv").replace("p", ".")
    )
    pwv = round(pwv, 1)
    return pwv

# PSG Radiance

In [ ]:
def read_psg_radiance_file(
    filename: str,
) -> lezargus.library.hint.LezargusSpectrum:
    """This function is a wrapper around the radiance file creation."""
    wave, total, earth = np.genfromtxt(filename, comments="#").T

    # We convert the wavelength from microns to meters.
    wavelength_si = lezargus.library.conversion.convert_units(
        value=wave, value_unit="um", result_unit="m"
    )

    # We attempt to get the zenith angle.
    zenith_angle = get_zenith_angle_from_filename(filename=filename)
    zenith_angle = lezargus.library.conversion.convert_units(
        zenith_angle, value_unit="deg", result_unit="rad"
    )
    airmass = lezargus.library.atmosphere.airmass(zenith_angle=zenith_angle)

    # We convert from the PSG units to the more typical units used in Lezaegus.
    psg_unit = astropy.units.Unit("W sr^-1 m^-2 um^-1")
    lz_unit = astropy.units.Unit("W m^-2 sr^-1 m^-1")
    total_si = lezargus.library.conversion.convert_units(
        value=total, value_unit=psg_unit, result_unit=lz_unit
    )
    total_si = np.array(total_si)

    # Creating the header information, where proper.
    header = {}
    header["LZI_INST"] = "PSG"
    header["LZO_NAME"] = "PSG Radiance"
    header["LZO_AIRM"] = lezargus.library.sanitize.fix_fits_header_value(
        input_data=airmass
    )

    # Creating the object.
    lz_psg_rad = lezargus.library.container.LezargusSpectrum(
        wavelength=wavelength_si,
        data=total_si,
        uncertainty=None,
        wavelength_unit="m",
        data_unit=lz_unit,
        flags=None,
        header=header,
    )
    return lz_psg_rad


def make_psg_radiance_wavelength_column(zenith_angle_str: str, pwv_str: str):
    """Create a table column for the provided PSG radiance wavelength."""
    # From the zenith angle and PWV string, we find the file and load it.
    filename = f"./base/atmosphere_psg/psg_cfg_telluric_{zenith_angle_str}_{pwv_str}_0p3-4p2um_rad.txt"
    psg_spectrum = read_psg_radiance_file(filename=filename)
    column = astropy.table.Column(
        psg_spectrum.wavelength, name="wavelength", dtype=float
    )
    return column


def make_psg_radiance_data_column(zenith_angle_str: str, pwv_str: str):
    """Create a table column for the provided PSG radiance data."""
    # From the zenith angle and PWV string, we find the file and load it.
    filename = f"./base/atmosphere_psg/psg_cfg_telluric_{zenith_angle_str}_{pwv_str}_0p3-4p2um_rad.txt"
    psg_spectrum = read_psg_radiance_file(filename=filename)
    # We derive the column name from the strings.
    pwv_number = (
        pwv_str.removeprefix("h2o").removesuffix("pwv").replace("p", ".")
    )
    column_name = f"{zenith_angle_str}_pwv{pwv_number}"
    column = astropy.table.Column(
        psg_spectrum.data,
        name=column_name,
        unit=psg_spectrum.data_unit,
        dtype=float,
    )
    return column

In [ ]:
# We do it over all zenith angles.
zenith_angle_string_list = ["za0", "za30", "za45", "za60"]
pwv_string_list = ["h2o0p5pwv", "h2o1p0pwv", "h2o2p0pwv", "h2o3p0pwv"]

wavelength_column = make_psg_radiance_wavelength_column(
    zenith_angle_str=zenith_angle_string_list[0], pwv_str=pwv_string_list[0]
)
# Getting the data columns. The wavelength is the first column.
table_columns = [wavelength_column]
for zenithdex in zenith_angle_string_list:
    for pwvdex in pwv_string_list:
        table_columns.append(
            make_psg_radiance_data_column(
                zenith_angle_str=zenithdex, pwv_str=pwvdex
            )
        )

# We need to build the table from the columns.
radiance_table = astropy.table.Table(table_columns)
radiance_table.write(
    f"./products/atmosphere_psg/psg_telluric_radiance.dat",
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in radiance_table.keys()},
    overwrite=True,
)

# PSG Transmission

In [ ]:
def read_psg_transmission_file(
    filename: str,
) -> lezargus.library.hint.LezargusSpectrum:
    """This function is a wrapper around the transmission file creation."""
    wave, total, h2o, co2, o3, n2o, co, ch4, o2, n2, rayleigh, cia = (
        np.genfromtxt(filename, comments="#").T
    )

    # We convert the wavelength from microns to meters.
    wave_meter = lezargus.library.conversion.convert_units(
        value=wave, value_unit="um", result_unit="m"
    )

    # We attempt to get the zenith angle.
    zenith_angle = get_zenith_angle_from_filename(filename=filename)
    zenith_angle = lezargus.library.conversion.convert_units(
        zenith_angle, value_unit="deg", result_unit="rad"
    )
    airmass = lezargus.library.atmosphere.airmass(zenith_angle=zenith_angle)

    # Creating the header information, where proper.
    header = {}
    header["LZI_INST"] = "PSG"
    header["LZO_NAME"] = "PSG Transmission"
    header["LZO_AIRM"] = lezargus.library.sanitize.fix_fits_header_value(
        input_data=airmass
    )

    # Creating the object.
    lz_psg_trans = lezargus.library.container.LezargusSpectrum(
        wavelength=wave_meter,
        data=total,
        uncertainty=None,
        wavelength_unit="m",
        data_unit="",
        flags=None,
        header=header,
    )
    return lz_psg_trans


def make_psg_transmission_wavelength_column(
    zenith_angle_str: str, pwv_str: str
):
    """Create a table column for the provided PSG transmission wavelength."""
    # From the zenith angle and PWV string, we find the file and load it.
    filename = f"./base/atmosphere_psg/psg_cfg_telluric_{zenith_angle_str}_{pwv_str}_0p3-4p2um_trn.txt"
    psg_spectrum = read_psg_transmission_file(filename=filename)
    column = astropy.table.Column(
        psg_spectrum.wavelength, name="wavelength", unit="m", dtype=float
    )
    return column


def make_psg_transmission_data_column(zenith_angle_str: str, pwv_str: str):
    """Create a table column for the provided PSG transmission data."""
    # From the zenith angle and PWV string, we find the file and load it.
    filename = f"./base/atmosphere_psg/psg_cfg_telluric_{zenith_angle_str}_{pwv_str}_0p3-4p2um_trn.txt"
    psg_spectrum = read_psg_transmission_file(filename=filename)
    # We derive the column name from the strings.
    pwv_number = (
        pwv_str.removeprefix("h2o").removesuffix("pwv").replace("p", ".")
    )
    column_name = f"{zenith_angle_str}_pwv{pwv_number}"
    column = astropy.table.Column(
        psg_spectrum.data, name=column_name, unit="", dtype=float
    )
    return column

In [ ]:
# We do it over all zenith angles.
zenith_angle_string_list = ["za0", "za30", "za45", "za60"]
pwv_string_list = ["h2o0p5pwv", "h2o1p0pwv", "h2o2p0pwv", "h2o3p0pwv"]

wavelength_column = make_psg_transmission_wavelength_column(
    zenith_angle_str=zenith_angle_string_list[0], pwv_str=pwv_string_list[0]
)
# Getting the data columns. The wavelength is the first column.
table_columns = [wavelength_column]
for zenithdex in zenith_angle_string_list:
    for pwvdex in pwv_string_list:
        table_columns.append(
            make_psg_transmission_data_column(
                zenith_angle_str=zenithdex, pwv_str=pwvdex
            )
        )

# We need to build the table from the columns.
radiance_table = astropy.table.Table(table_columns)
radiance_table.write(
    f"./products/atmosphere_psg/psg_telluric_transmission.dat",
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in radiance_table.keys()},
    overwrite=True,
)